<a href="https://colab.research.google.com/github/RohanKaushikan/CancerBioinformatics/blob/main/Differentially_Expressed_Genes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from scipy.stats import ttest_ind
from statsmodels.stats.multitest import multipletests

In [ ]:
df = pd.read_csv('/content/drive/Shareddrives/1:1_Rohan_Kaushikan/Dataset/rohan_merged_data_rna_seq_rsem.csv')

In [ ]:
df.head()

,Unnamed: 0,UBE2Q2P2,HMGB1P1,RNU12-2P,SSX9P,EZHIP,EFCAB8,SRP14P1,TRIM75P,SPATA31B1P,...,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3,Disease_Free_Status
0,TCGA-3C-AAAU,12.9316,52.1503,0.0000,0.0000,1.7233,0.3447,4.1359,0.6893,0.0,...,931.957,129.5920,1007.780,1658.500,258.4940,1208.370,3507.25,1894.930,1180.460,1:Recurred/Progressed
1,TCGA-3C-AALI,17.3790,69.7553,1.0875,0.5438,144.1000,4.8940,1.6313,0.5438,0.0,...,965.198,59.8151,448.613,1343.120,198.4770,603.589,5504.62,1318.650,406.743,0:DiseaseFree
2,TCGA-3C-AALJ,9.2294,154.2970,0.0000,0.0000,0.0000,2.7199,2.7199,0.0000,0.0,...,2531.280,35.3581,533.998,768.812,331.8220,532.185,5458.75,942.883,509.519,0:DiseaseFree
3,TCGA-3C-AALK,11.0799,143.8640,0.4137,0.0000,0.0000,2.0687,4.5511,0.0000,0.0,...,668.597,55.0269,437.733,863.881,175.4240,607.365,5691.35,781.134,700.869,NaN
4,TCGA-4H-AAAK,14.4298,84.2128,0.4255,0.0000,0.4255,2.9787,2.9787,0.0000,0.0,...,674.468,48.9362,424.255,1049.790,14.0426,775.745,4041.70,831.915,881.702,0:DiseaseFree


In [ ]:
df.shape

(1082, 20520)

In [ ]:
df_cleaned = df.dropna(subset=['Disease_Free_Status'])

In [ ]:
df_cleaned['Disease_Free_Status'].unique()

array(['1:Recurred/Progressed', '0:DiseaseFree'], dtype=object)

In [ ]:
X = df_cleaned.drop(['Disease_Free_Status'], axis=1)
y = df_cleaned['Disease_Free_Status']

In [ ]:
genes = []
p_values = []

In [ ]:
for gene in X.columns:
    # Assuming Disease_Free_Status is binary: '0:DiseaseFree' for non-diseased, '1:Recurred/Progressed' for diseased
    group1 = X[y == '0:DiseaseFree'].copy()
    group2 = X[y == '1:Recurred/Progressed'].copy()
    # Convert to numeric, coercing errors to NaNs
    g1 = pd.to_numeric(group1[gene], errors='coerce')
    g2 = pd.to_numeric(group2[gene], errors='coerce')

    # Perform t-test and ignore NaNs
    t_stat, p_val = ttest_ind(g1, g2, nan_policy='omit')

    # Store p-values and corresponding gene names
    p_values.append(p_val)
    genes.append(gene)

In [ ]:
corrected_p_values = multipletests(p_values, method='fdr_bh')[1]
print (corrected_p_values)

# Create a DataFrame with genes and their p-values
results = pd.DataFrame({
    'Gene': genes,
    'p_value': corrected_p_values
})